### 1. Cài đặt và nhập thư viện


In [7]:
# Kiểm tra và cài đặt thư viện nếu thiếu
def import_libraries():
    try:
        global selenium, selenium_stealth
        import selenium
        import selenium_stealth
        print("✅ Import thư viện thành công.")
        return True
    except ImportError as e:
        print(f"⚠️ Thiếu thư viện {e.name}, tiến hành cài đặt...")
        return False

# Kiểm tra và cài đặt nếu cần
if not import_libraries():
    subprocess.check_call([sys.executable, "-m", "pip", "install", "selenium", "selenium-stealth"])
    print("✅ Cài đặt hoàn tất. Đang kiểm tra lại...")
    import_libraries()
    
# ==== Import các thư viện cần thiết ====
import sys
import subprocess
import time
import random
import csv
import os
import threading
from concurrent.futures import ThreadPoolExecutor, as_completed
from threading import Lock
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
from selenium_stealth import stealth


✅ Import thư viện thành công.


<div style="font-size: 13px">

| 📚 Thư viện                              | 📝 Mô tả chi tiết                                                                                                                                |
| ---------------------------------------- | ------------------------------------------------------------------------------------------------------------------------------------------------ |
| selenium                                 | Thư viện điều khiển trình duyệt tự động, cho phép mô phỏng hành vi người dùng như click, nhập liệu, lấy dữ liệu từ trang web.                    |
| selenium_stealth                         | Dùng để "ngụy trang" trình duyệt do Selenium tạo ra, giúp tránh bị website phát hiện là bot (ví dụ như qua navigator.webdriver).                 |
| concurrent.futures.ThreadPoolExecutor    | Cho phép chạy nhiều tác vụ song song bằng luồng (thread), tăng tốc xử lý trong các tác vụ như thu thập dữ liệu từ nhiều trang web cùng lúc.      |
| threading.Lock                           | Dùng để đồng bộ giữa các luồng khi truy cập tài nguyên chung (như ghi file), tránh lỗi do nhiều luồng truy cập cùng lúc.                         |
| queue.Queue                              | Một hàng đợi an toàn khi dùng đa luồng (thread-safe), rất hữu ích khi truyền dữ liệu giữa các luồng trong quá trình thu thập hoặc xử lý dữ liệu. |
| WebDriverWait, expected_conditions (EC)  | Dùng để **chờ một điều kiện xảy ra**, ví dụ: chờ cho đến khi một phần tử xuất hiện trước khi thực hiện hành động tiếp theo.                      |
| NoSuchElementException, TimeoutException | Bắt các lỗi phổ biến khi Selenium không tìm thấy phần tử hoặc chờ quá lâu, giúp chương trình không bị crash khi gặp trang web thay đổi bất ngờ.  |

</div>


### 2. Cấu hình các tham số


In [8]:
# Cấu hình
START_ROW = 42000
END_ROW = 49303
INPUT_FILE = "urls.csv"
OUTPUT_FILE = "details.csv"
NUM_THREADS = 3  
MAX_RETRIES = 3 
lock = Lock()

output_fields = [
    "Tiêu đề", "Địa điểm", "Mức giá", "Diện tích", "Số phòng ngủ",
    "Số phòng tắm, vệ sinh", "Số tầng", "Hướng nhà", "Hướng ban công",
    "Đường vào", "Mặt tiền", "Pháp lý", "Nội thất", "Ngày đăng", "URL"
]

<div style="font-size: 13px">

| 🧩 Biến / Hằng | 📝 Mô tả                                                                                                                                                                                                                  |
| -------------- | ------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------- |
| NUM_THREADS    | Số luồng (threads) sẽ được sử dụng để xử lý song song. Tăng số này giúp chương trình nhanh hơn khi thu thập dữ liệu nhiều trang.                                                                                          |
| MAX_RETRIES    | Số lần thử lại tối đa nếu một thao tác (ví dụ như tải trang hoặc tìm phần tử) bị lỗi hoặc timeout.                                                                                                                        |
| lock = Lock()  | Tạo một **khóa luồng (thread lock)** để đảm bảo rằng **chỉ một luồng được thực thi đoạn mã quan trọng tại một thời điểm**, giúp tránh xung đột khi nhiều luồng cùng ghi vào một tài nguyên như file hoặc biến dùng chung. |
| BASE_URL       | URL gốc của trang web cần thu thập dữ liệu. Có thể thay đổi để thu thập các loại bất động sản khác nhau như nhà cho thuê, đất nền,...                                                                                     |
| START_PAGE     | Trang bắt đầu thu thập dữ liệu.                                                                                                                                                                                           |
| END_PAGE       | Trang kết thúc thu thập dữ liệu.                                                                                                                                                                                          |
| OUTPUT_FILE    | Tên file CSV dùng để lưu dữ liệu đã thu thập.                                                                                                                                                                             |

</div>


### 3. Cấu hình Selenium


In [9]:
def setup_driver():
    """Khởi tạo trình duyệt với selenium-stealth để tránh bị chặn."""
    options = Options()
    options.add_argument("--disable-blink-features=AutomationControlled")
    options.add_argument("--headless")  # Chạy không giao diện
    options.add_argument("--disable-gpu")
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")
    options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.51 Safari/537.36")

    driver = webdriver.Chrome(options=options)

    # Dùng selenium-stealth để tránh bị phát hiện là bot
    stealth(driver,
            languages=["en-US", "en"],
            vendor="Google Inc.",
            platform="Win32",
            webgl_vendor="Intel Inc.",
            renderer="Intel Iris OpenGL Engine",
            fix_hairline=True)

    driver.delete_all_cookies()
    return driver

<div style="font-size: 13px">

| Tham số                                          | Mô tả                                                                                                                |
| ------------------------------------------------ | -------------------------------------------------------------------------------------------------------------------- |
| 🖥️ --headless                                    | Chạy trình duyệt ẩn                                                                                                  |
| 🚫 --disable-blink-features=AutomationControlled | Giảm khả năng bị phát hiện là bot bằng cách vô hiệu hóa cơ chế kiểm tra tự động của Chrome.                          |
| 🎮 --disable-gpu                                 | Vô hiệu hóa GPU, tránh lỗi khi chạy trên máy chủ không có giao diện đồ họa.                                          |
| 🎨 --enable-unsafe-webgl                         | Kích hoạt WebGL ngay cả khi trình duyệt cho rằng nó không an toàn, giúp tránh bị phát hiện thông qua dấu vết đồ họa. |
| 🔓 --no-sandbox                                  | Loại bỏ cơ chế "sandbox" của Chrome, cần thiết khi chạy trên hệ điều hành Linux không có giao diện đồ họa.           |
| 💾 --disable-dev-shm-usage                       | Tránh lỗi hết bộ nhớ chia sẻ (/dev/shm) trên hệ thống Linux khi Chrome sử dụng quá nhiều bộ nhớ.                     |
| 🕵️‍♂️ Giả lập User-Agent                            | Giúp trình duyệt giả lập như một người dùng thật để tránh bị phát hiện là bot.                                       |
| 🎭 --disable-extensions                          | Vô hiệu hóa tất cả tiện ích mở rộng để giảm khả năng bị phát hiện là bot.                                            |
| 🧹 --disable-popup-blocking                      | Ngăn trình duyệt chặn pop-up, hữu ích khi cần tương tác với các cửa sổ bật lên.                                      |
| 🔄 --remote-debugging-port=9222                  | Cho phép gỡ lỗi từ xa, hữu ích khi cần theo dõi hành vi của trình duyệt.                                             |

</div>


### 4. Định nghĩa các hàm thu thập dữ liệu


Trong phần này, chúng ta sẽ xây dựng hai hàm chính để thu thập dữ liệu từ website bất động sản:

- `scrape_page(page, retries=0)`: Thu thập dữ liệu từ một trang cụ thể với cơ chế retry.
- `crawl_pages_concurrently(start_page, end_page, output_file)`: Chạy nhiều tác vụ thu thập song song bằng đa luồng để tăng tốc độ.


#### 4.1 Hàm scrape_page(page, retries=0)


In [10]:
def scrape_detail(url, retries=0):
    """Thu thập thông tin chi tiết từ một bài đăng cụ thể dựa theo icon class."""
    driver = setup_driver()

    # Mapping icon class -> field name
    icon_map = {
        "re__icon-money": "Mức giá",
        "re__icon-size": "Diện tích",
        "re__icon-bedroom": "Số phòng ngủ",
        "re__icon-bath": "Số phòng tắm, vệ sinh",
        "re__icon-apartment": "Số tầng",
        "re__icon-front-view": "Hướng nhà",
        "re__icon-private-house": "Hướng ban công",
        "re__icon-road": "Đường vào",
        "re__icon-home": "Mặt tiền",
        "re__icon-document": "Pháp lý",
        "re__icon-interior": "Nội thất"
    }

    # Khởi tạo dữ liệu với tất cả trường = None
    data = {"url": url}
    for field in icon_map.values():
        data[field] = None

    try:
        driver.get(url)
        wait = WebDriverWait(driver, 10, poll_frequency=0.5)
        info_items = wait.until(EC.presence_of_all_elements_located(
            (By.CSS_SELECTOR, "div.re__pr-other-info-display div.re__pr-specs-content-item")
        ))

        for item in info_items:
            try:
                icon_element = item.find_element(By.CSS_SELECTOR, "span.re__pr-specs-content-item-icon i")
                icon_class = icon_element.get_attribute("class")

                # Lấy tên trường từ icon_class
                for key in icon_map:
                    if key in icon_class:
                        field_name = icon_map[key]
                        value = item.find_element(By.CLASS_NAME, "re__pr-specs-content-item-value").text.strip()
                        data[field_name] = value
                        break
            except NoSuchElementException:
                continue

        return data
    except Exception as e:
        if retries < MAX_RETRIES:
            print(f"⚠️ Lỗi crawl {url}, thử lại lần {retries + 1}...")
            time.sleep(random.uniform(2, 5))
            return scrape_detail(url, retries + 1)
        else:
            print(f"❌ Bỏ qua URL {url} do lỗi: {e}")
            return None
    finally:
        driver.quit()

- **Tham số**:

  - `page`: Số trang cần thu thập.
  - `retries`: Số lần thử lại nếu xảy ra lỗi (mặc định = 0).

- **Cơ chế xử lý**:

  - Dùng Selenium và `WebDriverWait` để chờ trang load đủ.
  - Lấy các thông tin: tiêu đề, giá, diện tích, phòng ngủ, vệ sinh, vị trí, ngày đăng.
  - Nếu không có dữ liệu hoặc bị lỗi, tự động thử lại tới `MAX_RETRIES`.

- **Kết quả trả về**:
  - Trả về danh sách dữ liệu nếu thành công.
  - Trả về `None` nếu không còn dữ liệu hoặc lỗi sau nhiều lần thử.


#### 4.2 Hàm crawl_pages_concurrently(start_page, end_page, output_file)


In [11]:
def crawl_details_concurrently(input_file, output_file):
    """\
    Đọc danh sách URL và thông tin cơ bản, crawl thêm chi tiết và ghi vào file.
    Sau khi thành công, xóa bản ghi khỏi file input ngay lập tức.
    """
    with open(input_file, newline='', encoding='utf-8-sig') as csvfile:
        reader = csv.DictReader(csvfile)
        rows = [row for row in reader if row.get("URL")]

    if not os.path.exists(output_file) or os.stat(output_file).st_size == 0:
        with open(output_file, "w", newline="", encoding="utf-8-sig") as f:
            writer = csv.DictWriter(f, fieldnames=output_fields)
            writer.writeheader()

    # Lấy con số dòng bắt đầu và kết thúc
    rows_to_process = rows[START_ROW - 1: END_ROW]  # Chú ý chỉ số bắt đầu từ 0
    total = len(rows)

    def worker(index_row):
        index, row = index_row
        url = row["URL"]
        detail_data = scrape_detail(url)
        if detail_data:
            full_row = {
                "Tiêu đề": row.get("Tiêu đề", ""),
                "Địa điểm": row.get("Địa điểm", ""),
                "Ngày đăng": row.get("Ngày đăng", ""),
                "URL": url,
                **detail_data
            }
            ordered_row = {field: full_row.get(field, "") for field in output_fields}
            with lock:
                with open(output_file, "a", newline="", encoding="utf-8-sig") as f:
                    writer = csv.DictWriter(f, fieldnames=output_fields)
                    writer.writerow(ordered_row)
            print(f"✅ Đã lưu bản ghi {index}/{total}")

    indexed_rows = list(enumerate(rows_to_process, start=START_ROW))
    with ThreadPoolExecutor(max_workers=NUM_THREADS) as executor:
        futures = [executor.submit(worker, item) for item in indexed_rows]
        for future in as_completed(futures):
            future.result()

    print(f"✅ Crawl chi tiết hoàn tất. Dữ liệu được lưu tại `{output_file}`.")


- **Tham số**:

  - `start_page`, `end_page`: khoảng trang cần crawl.
  - `output_file`: đường dẫn file CSV để lưu dữ liệu.

- **Cơ chế hoạt động**:

  - Dùng `queue.Queue` để phân phối các trang cho các luồng.
  - Mỗi luồng sẽ lấy 1 trang từ queue, gọi `scrape_page`, và ghi kết quả ra file.
  - Dùng `threading.Lock` để tránh ghi file cùng lúc gây lỗi.

- **Lợi ích**:
  - Tăng tốc độ crawl nhờ chạy song song.
  - Có thể điều chỉnh số luồng bằng `NUM_THREADS` để phù hợp tài nguyên hệ thống.


### 5. Chạy trình thu thập dữ liệu


In [ ]:
# Ghi tiêu đề CSV
with open(OUTPUT_FILE, "w", newline="", encoding="utf-8-sig") as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(output_fields)

# Gọi hàm crawl
crawl_details_concurrently(INPUT_FILE, OUTPUT_FILE)

✅ Đã lưu bản ghi 42000/49303
✅ Đã lưu bản ghi 42002/49303
✅ Đã lưu bản ghi 42001/49303
✅ Đã lưu bản ghi 42003/49303
✅ Đã lưu bản ghi 42005/49303
✅ Đã lưu bản ghi 42004/49303
⚠️ Lỗi crawl https://batdongsan.com.vn/ban-nha-rieng-duong-nguyen-van-lac-phuong-19/hang-ngop-binh-thanh-gia-2-ty-788-ban-gap-hem-oto-thong-tu-phia-48m2-shr-3pn-pr42733122, thử lại lần 1...
⚠️ Lỗi crawl https://batdongsan.com.vn/ban-nha-rieng-duong-hong-ha-phuong-phuc-xa/-moi-hien-dai-ba-dinh-thang-may-o-to-tranh-vi-tri-qua-dep-ve-o-luon-pr42378396, thử lại lần 1...
⚠️ Lỗi crawl https://batdongsan.com.vn/ban-nha-rieng-duong-xo-viet-nghe-tinh-phuong-21/ban-gap-48m2-hem-xe-hoi-chung-cu-my-duc-221-p21-binh-thanh-shr-2ty750tl-pr41859033, thử lại lần 1...
⚠️ Lỗi crawl https://batdongsan.com.vn/ban-nha-rieng-duong-nguyen-van-lac-phuong-19/hang-ngop-binh-thanh-gia-2-ty-788-ban-gap-hem-oto-thong-tu-phia-48m2-shr-3pn-pr42733122, thử lại lần 2...
⚠️ Lỗi crawl https://batdongsan.com.vn/ban-nha-rieng-duong-xo-viet-nghe-tinh-ph